# Visualisation of masks for DICOM files in the dataset.

Observations
- These were different from those of NIFTI files converted using dicom2nifti which were rotated and indexed in a reverse order wrt slices
- These correspond much better to the annotation boxes
- There are a few boxes that are incorrectly labelled. Sequences with double-digit number of slices have an additional '0' in the simple paths dataset

In [1]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg itkwidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
from matplotlib import patches, text, patheffects

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

import re
from tqdm import tqdm
import itertools
from functools import wraps
import ipywidgets

tqdm.pandas()

c:\ProgramData\Anaconda3\envs\mri_project\lib\site-packages\nilearn\__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [3]:
DATASET_PATH = '../../Simple Path Dataset/manifest-1654812109500/'

SAVE_PATH = '../../Pre_minus_post'

os.makedirs(SAVE_PATH, exist_ok=True)

In [4]:
annotation_boxes = pd.read_excel("../../Simple Path Dataset/Annotation_Boxes.xlsx").set_index("Patient ID")

dataset = pd.read_excel('../../Simple Path Dataset/Breast-Cancer-MRI-filepath_filename-mapping.xlsx')

dataset = dataset.iloc[:, :4]

In [5]:
dataset

,sop_instance_UID,original_path_and_filename,classic_path,descriptive_path
0,1.3.6.1.4.1.14519.5.2.1.1805789812895034139917...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...
1,1.3.6.1.4.1.14519.5.2.1.4903237729147735321973...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...
2,1.3.6.1.4.1.14519.5.2.1.3061160038794820079325...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...
3,1.3.6.1.4.1.14519.5.2.1.1574717199045785031549...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...
4,1.3.6.1.4.1.14519.5.2.1.2594404476894572978078...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...
...,...,...,...,...
773121,1.3.6.1.4.1.14519.5.2.1.2393425910452664915158...,DICOM_Images/Breast_MRI_922/T1/Breast_MRI_922_...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...
773122,1.3.6.1.4.1.14519.5.2.1.1921416620775060655567...,DICOM_Images/Breast_MRI_922/T1/Breast_MRI_922_...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...
773123,1.3.6.1.4.1.14519.5.2.1.1148085546624000943273...,DICOM_Images/Breast_MRI_922/T1/Breast_MRI_922_...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...
773124,1.3.6.1.4.1.14519.5.2.1.1929308890572593698573...,DICOM_Images/Breast_MRI_922/T1/Breast_MRI_922_...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...


In [6]:
dataset['original_path_and_filename'][0].split('/')[1:3]

['Breast_MRI_001', 'post_1']

In [7]:
dataset[['patient', 'sequence']] = dataset.progress_apply(lambda x: x['original_path_and_filename'].split('/')[1:3], result_type = 'expand', axis = 1)

100%|██████████| 773126/773126 [00:17<00:00, 44850.96it/s] 


In [8]:
dataset = dataset[dataset['sequence'].isin(['pre', 'post_1'])].reset_index(drop = True)
dataset['slice_number'] = dataset['classic_path'].apply(lambda x: x.split('/')[-1].split('-')[-1].replace('.dcm', ""))

In [9]:
dataset

,sop_instance_UID,original_path_and_filename,classic_path,descriptive_path,patient,sequence,slice_number
0,1.3.6.1.4.1.14519.5.2.1.1805789812895034139917...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,Breast_MRI_001,post_1,001
1,1.3.6.1.4.1.14519.5.2.1.4903237729147735321973...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,Breast_MRI_001,post_1,002
2,1.3.6.1.4.1.14519.5.2.1.3061160038794820079325...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,Breast_MRI_001,post_1,003
3,1.3.6.1.4.1.14519.5.2.1.1574717199045785031549...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,Breast_MRI_001,post_1,004
4,1.3.6.1.4.1.14519.5.2.1.2594404476894572978078...,DICOM_Images/Breast_MRI_001/post_1/Breast_MRI_...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,Breast_MRI_001,post_1,005
...,...,...,...,...,...,...,...
314391,1.3.6.1.4.1.14519.5.2.1.1620326542824048399612...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,Breast_MRI_922,pre,170
314392,1.3.6.1.4.1.14519.5.2.1.1729373208009133770295...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,Breast_MRI_922,pre,171
314393,1.3.6.1.4.1.14519.5.2.1.1303194464206675272654...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,Breast_MRI_922,pre,172
314394,1.3.6.1.4.1.14519.5.2.1.3292134909528830152433...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,Breast_MRI_922,pre,173


## Extracting and saving bounding boxes for all patients


In [10]:
def relu(x):
    return np.maximum(x, 0)

In [11]:
patients = dataset.patient.unique()

In [12]:
os.makedirs(SAVE_PATH, exist_ok= True)

errorfiles = []

for patient_id in tqdm(patients):

    # making directory for patient
    # patient_dir = os.path.join(SAVE_PATH, patient_id)
    # os.makedirs(patient_dir, exist_ok= True)

    patient_1 = dataset[dataset['patient'] == patient_id]
    patient_1['slice_number'] = patient_1['slice_number'].astype('int')
    patient_1_pre = patient_1[patient_1.sequence == 'pre'].reset_index(drop = True).set_index('slice_number')
    patient_1_post = patient_1[patient_1.sequence == 'post_1'].reset_index(drop = True).set_index('slice_number')


    # getting bounding boxes
    x = annotation_boxes.loc[patient_id]
    row1 = x['Start Row']
    row2 = x['End Row']

    col1 = x['Start Column']
    col2 = x['End Column']

    slice1 = x['Start Slice']
    slice2 = x['End Slice']

    # middle slice
    mid_slice = (slice1 + slice2)//2                                     # num slices - current slice

    dicom_pre_path = patient_1_pre.loc[mid_slice].classic_path                              # read dicom slice for precontrast
    dicom_post_path = patient_1_post.loc[mid_slice].classic_path                            # read dicom slice for postcontrast

    try:
        precontrast_data = dicom.read_file(DATASET_PATH + dicom_pre_path).pixel_array               #get pixel data for precontrast
        postcontrast_data = dicom.read_file(DATASET_PATH + dicom_post_path).pixel_array             #get pixel data for postcontrast
    
    except:
        print("\nERROR WITH {}".format(patient_id))
        print(dicom_pre_path)
        print(dicom_post_path)

        errorfiles.append([patient_id, dicom_pre_path, dicom_post_path])

    # precontrast_data = relu(precontrast_data)                                                   # apply relu here (can comment out)
    # postcontrast_data = relu(postcontrast_data)                                                 # apply relu here (can comment out)

    post_minus_pre = postcontrast_data - precontrast_data                                       # differenced
    pre_minus_post = precontrast_data - postcontrast_data                                       # differenced

    fig, ax = plt.subplots(1, 4, figsize = (27, 9))                                             # plotting


    # precontrast
    fig.suptitle("{}\n\n{}, Slice Number: {}\nThe red box denotes the bounding box for the tumour as per the dataset".format(patient_id, 'Middle Slice', mid_slice))
    ax[0].imshow(precontrast_data, cmap = 'gray')
    ax[0].set_title("Precontrast")
    ax[0].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

    # postcontrast
    ax[1].imshow(postcontrast_data, cmap = 'gray')
    ax[1].set_title("Postcontrast-1")
    ax[1].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

    # difference
    ax[2].imshow(post_minus_pre, cmap = 'gray')
    ax[2].set_title("Differenced: Postcontrast1 - Precontrast")
    ax[2].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

    ax[3].imshow(pre_minus_post, cmap = 'gray')
    ax[3].set_title("Differenced: Precontrast - Postcontrast1")
    ax[3].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))



    # saving
    final_path = os.path.join(SAVE_PATH,  f"{patient_id}.png")
    fig.savefig(final_path, facecolor = 'white')


    plt.close()

  0%|          | 0/922 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\mri_project\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
 13%|█▎        | 119/922 [00:41<04:22,  3.06it/s]


ERROR WITH Breast_MRI_120
Duke-Breast-Cancer-MRI/Breast_MRI_120/1.3.6.1.4.1.14519.5.2.1.211850134288720912089409947004890026375/1.3.6.1.4.1.14519.5.2.1.182581940488565993251476651992238460998/1-046.dcm
Duke-Breast-Cancer-MRI/Breast_MRI_120/1.3.6.1.4.1.14519.5.2.1.211850134288720912089409947004890026375/1.3.6.1.4.1.14519.5.2.1.123737368708979440735273631651698929376/1-046.dcm


 32%|███▏      | 298/922 [01:43<03:30,  2.97it/s]


ERROR WITH Breast_MRI_299
Duke-Breast-Cancer-MRI/Breast_MRI_299/1.3.6.1.4.1.14519.5.2.1.36906871641716161784732996276122626294/1.3.6.1.4.1.14519.5.2.1.218216529583049019480450611266793573286/1-025.dcm
Duke-Breast-Cancer-MRI/Breast_MRI_299/1.3.6.1.4.1.14519.5.2.1.36906871641716161784732996276122626294/1.3.6.1.4.1.14519.5.2.1.157193435424726054777601183184809809756/1-025.dcm


 35%|███▍      | 321/922 [01:51<03:08,  3.19it/s]

...

 93%|█████████▎| 862/922 [05:04<00:20,  2.93it/s]


ERROR WITH Breast_MRI_863
Duke-Breast-Cancer-MRI/Breast_MRI_863/1.3.6.1.4.1.14519.5.2.1.14693917311997896050005042418199958786/1.3.6.1.4.1.14519.5.2.1.181295351944923651503104313786928244114/1-043.dcm
Duke-Breast-Cancer-MRI/Breast_MRI_863/1.3.6.1.4.1.14519.5.2.1.14693917311997896050005042418199958786/1.3.6.1.4.1.14519.5.2.1.167862497583042435511519403145811775877/1-043.dcm


 98%|█████████▊| 905/922 [05:22<00:08,  2.11it/s]


ERROR WITH Breast_MRI_906
Duke-Breast-Cancer-MRI/Breast_MRI_906/1.3.6.1.4.1.14519.5.2.1.219026347972697075635085423072976723322/1.3.6.1.4.1.14519.5.2.1.218315360971086161626572686172618858161/1-035.dcm
Duke-Breast-Cancer-MRI/Breast_MRI_906/1.3.6.1.4.1.14519.5.2.1.219026347972697075635085423072976723322/1.3.6.1.4.1.14519.5.2.1.57685914747423128458494088294665709813/1-035.dcm


100%|██████████| 922/922 [05:28<00:00,  2.81it/s]


## Erroneous cases

In [13]:
erroneous = pd.DataFrame(errorfiles, columns = ['patient_id', 'precontrast', 'postcontrast']).drop_duplicates()

In [14]:
erroneous['precontrast'][0].split('/')[-1].split('-')

['1', '046.dcm']

### Loop for erroneous cases

In [15]:
def double_digit(path):
    '''Function to process sequences with <100 dicom images'''
    path = path.split('/')
    filename = path[-1].split('-')
    filename[1] = filename[1][1:]
    filename = '-'.join(filename)
    path[-1] = filename
    path = '/'.join(path)

    return path

In [16]:
double_digit(erroneous['precontrast'][0])

'Duke-Breast-Cancer-MRI/Breast_MRI_120/1.3.6.1.4.1.14519.5.2.1.211850134288720912089409947004890026375/1.3.6.1.4.1.14519.5.2.1.182581940488565993251476651992238460998/1-46.dcm'

In [17]:
for patient_id in tqdm(erroneous.patient_id.unique()):

    # making directory for patient
    # patient_dir = os.path.join(SAVE_PATH, patient_id)
    # os.makedirs(patient_dir, exist_ok= True)

    patient_1 = dataset[dataset['patient'] == patient_id]
    patient_1['slice_number'] = patient_1['slice_number'].astype('int')
    patient_1_pre = patient_1[patient_1.sequence == 'pre'].reset_index(drop = True).set_index('slice_number')
    patient_1_post = patient_1[patient_1.sequence == 'post_1'].reset_index(drop = True).set_index('slice_number')


    # getting bounding boxes
    x = annotation_boxes.loc[patient_id]
    row1 = x['Start Row']
    row2 = x['End Row']

    col1 = x['Start Column']
    col2 = x['End Column']

    slice1 = x['Start Slice']
    slice2 = x['End Slice']

    # middle slice
    mid_slice = (slice1 + slice2)//2                                     # num slices - current slice


    slice_name = " ".join('middle_slice'.split('_')).title()                                       # printing info

    dicom_pre_path = patient_1_pre.loc[mid_slice].classic_path                              # read dicom slice for precontrast
    dicom_post_path = patient_1_post.loc[mid_slice].classic_path                            # read dicom slice for postcontrast


    # CHANGES OVER OLD LOOP
    dicom_pre_path = double_digit(dicom_pre_path)
    dicom_post_path = double_digit(dicom_post_path)

    try:
        precontrast_data = dicom.read_file(DATASET_PATH + dicom_pre_path).pixel_array               #get pixel data for precontrast
        postcontrast_data = dicom.read_file(DATASET_PATH + dicom_post_path).pixel_array             #get pixel data for postcontrast
    
    except:
        print("\nERROR WITH {}".format(patient_id))
        print(dicom_pre_path)
        print(dicom_post_path)

        errorfiles.append([patient_id, dicom_pre_path, dicom_post_path])

    # precontrast_data = relu(precontrast_data)                                                   # apply relu here (can comment out)
    # postcontrast_data = relu(postcontrast_data)                                                 # apply relu here (can comment out)

    diff_contrast = postcontrast_data - precontrast_data                                        # differenced

    fig, ax = plt.subplots(1, 4, figsize = (27, 9))                                             # plotting


    # precontrast
    fig.suptitle("{}\n\n{}, Slice Number: {}\nThe red box denotes the bounding box for the tumour as per the dataset".format(patient_id, 'Middle Slice', mid_slice))
    ax[0].imshow(precontrast_data, cmap = 'gray')
    ax[0].set_title("Precontrast")
    ax[0].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

    # postcontrast
    ax[1].imshow(postcontrast_data, cmap = 'gray')
    ax[1].set_title("Postcontrast-1")
    ax[1].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

    # difference
    ax[2].imshow(post_minus_pre, cmap = 'gray')
    ax[2].set_title("Differenced: Postcontrast1 - Precontrast")
    ax[2].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

    ax[3].imshow(pre_minus_post, cmap = 'gray')
    ax[3].set_title("Differenced: Precontrast - Postcontrast1")
    ax[3].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))



    # saving
    final_path = os.path.join(SAVE_PATH,  f"{patient_id}.png")
    fig.savefig(final_path, facecolor = 'white')


    plt.close()

  0%|          | 0/11 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\mri_project\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 11/11 [00:03<00:00,  2.96it/s]
